In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import tabulate as tb

In [16]:
TICKER = "GOOGL"
START_DATE = "2000-01-01"
END_DATE = None
TIMEFRAME = "1d"    # 1d, 1h, 15m, 5m

In [17]:
df = yf.download(
    TICKER, 
    start=START_DATE,
    end=END_DATE,
    interval=TIMEFRAME,
    progress=False
)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

df.dropna(inplace=True)
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

C:\Users\dzied\AppData\Local\Temp\ipykernel_19872\3251543518.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


+---------------------+---------+--------+--------+--------+-------------+
| Date                |   Close |   High |    Low |   Open |      Volume |
|---------------------+---------+--------+--------+--------+-------------|
| 2025-10-13 00:00:00 |  244.15 | 244.5  | 239.71 | 240.21 | 2.4995e+07  |
| 2025-10-14 00:00:00 |  245.45 | 247.12 | 240.51 | 241.23 | 2.21116e+07 |
| 2025-10-15 00:00:00 |  251.03 | 252.11 | 245.99 | 247.25 | 2.70077e+07 |
| 2025-10-16 00:00:00 |  251.46 | 256.96 | 250.1  | 251.77 | 2.79972e+07 |
| 2025-10-17 00:00:00 |  253.3  | 254.22 | 247.81 | 250.76 | 2.95861e+07 |
+---------------------+---------+--------+--------+--------+-------------+


In [18]:
df["rsi"] = ta.rsi(df["Close"], length=14)
macd = ta.macd(df["Close"])
df = df.join(macd)

bbands = ta.bbands(df["Close"], length=20)
df = df.join(bbands)

df["ema_10"] = ta.ema(df["Close"], length=10)
df["ema_20"] = ta.ema(df["Close"], length=20)
df["ema_50"] = ta.ema(df["Close"], length=50)
df["atr"] = ta.atr(df["High"], df["Low"], df["Close"], length=14)
stoch = ta.stoch(df["High"], df["Low"], df["Close"])
df["stoch_k"] = stoch.iloc[:, 0]
df["stoch_d"] = stoch.iloc[:, 1]
df["returns"] = df["Close"].pct_change()

df.dropna(inplace=True)

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

df.to_csv(f"./../data/{TICKER}.csv")

+---------------------+---------+---------+---------+---------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+----------+-----------+-----------+-------------+
| Date                |   Close |    High |     Low |    Open |      Volume |     rsi |   MACD_12_26_9 |   MACDh_12_26_9 |   MACDs_12_26_9 |   BBL_20_2.0_2.0 |   BBM_20_2.0_2.0 |   BBU_20_2.0_2.0 |   BBB_20_2.0_2.0 |   BBP_20_2.0_2.0 |   ema_10 |   ema_20 |   ema_50 |      atr |   stoch_k |   stoch_d |     returns |
|---------------------+---------+---------+---------+---------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+----------+-----------+-----------+-------------|
| 2004-10-28 00:00:00 | 4.8042  | 4.83129 | 4.